In [ ]:
!sudo apt-get update && sudo apt-get install -y apt-transport-https
!curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://apt.kubernetes.io/ kubernetes-xenial main" | sudo tee -a /etc/apt/sources.list.d/kubernetes.list
!sudo apt-get update
!sudo apt-get install -y kubectl

In [ ]:
!curl https://raw.githubusercontent.com/kubernetes/helm/master/scripts/get | bash

In [ ]:
!sudo apt-get install gir1.2-glib-2.0

In [ ]:
!pip install --upgrade dask[complete] dask-kubernetes dask-gateway 

**RESTART YOUR KERNEL**

In [1]:
!az login >> /dev/null

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BAPY86UDP to authenticate.
Failed to authenticate '{'additional_properties': {'displayName': 'Chevron', 'domains': ['pasadenarefining.com', 'chevronstores.com', 'caltexgo.com', 'caltex.com.vn', 'chevronpac.com', 'chevronlummusglobal.com', 'chevronuktr.com', 'xponentcard.com', 'chevronenergy.com', 'chevronlubricants.com', 'cvxarea52.com', 'deepstar.org', 'arteco-coolants.eu', 'chevronpartnershipprograms.com', 'chevronadvocacynetwork.com', 'anlngservices.com', 'catchlightenergy.com', 'sasolchevron.com', 'caltex.com', 'richmondrefinery.com', 'chevronapc.onmicrosoft.com', 'chevroneur.onmicrosoft.com', 'chevrontexaco.net', 'chevrontexaco.com', 'tengizchevroil.net', 'tengizchevroil.com', 'tco.tengizchevroil.net', 'chevron.mail.onmicrosoft.com', 'chevron.onmicrosoft.com', 'ct.chevrontexaco.net', 'chevron.net', 'chevron.com'], 'countryCode': 'US', 'tenantCategory': 'Home'}, 'id': '/tenants/fd799

In [2]:
!az account show

{
  "environmentName": "AzureCloud",
  "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "id": "6560575d-fa06-4e7d-95fb-f962e74efd7a",
  "isDefault": true,
  "managedByTenants": [
    {
      "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
    }
  ],
  "name": "Data4ML PM",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "copeters@microsoft.com",
    "type": "user"
  }
}


In [3]:
import os
os.environ['RG']  = 'cody-aks-rg'
os.environ['AKS'] = 'dasky3'
os.environ['LOC'] = 'westus2'
os.environ['VM']  = 'STANDARD_DS12_V2'
os.environ['MIN'] = '3'
os.environ['MAX'] = '100'

# used for dask-gateway
os.environ['DG']  = 'dask-gateway'
os.environ['DGV'] = '0.6.1'

In [4]:
!az group create -n $RG -l $LOC

{
  "id": "/subscriptions/6560575d-fa06-4e7d-95fb-f962e74efd7a/resourceGroups/cody-image-rg",
  "location": "westus2",
  "managedBy": null,
  "name": "cody-image-rg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [9]:
!az aks create -n $AKS -g $RG --generate-ssh-keys --node-vm-size $VM --enable-cluster-autoscaler --min-count $MIN --max-count $MAX

 - Finished ..principal creation[##################################]  100.0000%{
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "availabilityZones": null,
      "count": 3,
      "enableAutoScaling": true,
      "enableNodePublicIp": null,
      "maxCount": 100,
      "maxPods": 110,
      "minCount": 3,
      "name": "nodepool1",
      "nodeLabels": null,
      "nodeTaints": null,
      "orchestratorVersion": "1.15.10",
      "osDiskSizeGb": 100,
      "osType": "Linux",
      "provisioningState": "Succeeded",
      "scaleSetEvictionPolicy": null,
      "scaleSetPriority": null,
      "tags": null,
      "type": "VirtualMachineScaleSets",
      "vmSize": "STANDARD_DS12_V2",
      "vnetSubnetId": null
    }
  ],
  "apiServerAccessProfile": null,
  "dnsPrefix": "cody-image-cody-image-rg-656057",
  "enablePodSecurityPolicy": null,
  "enableRbac": true,
  "fqdn": "cody-image-cody-image-rg-656057-9c27a87c.hcp.westus2.azmk8s.io",
  "id": "/subscriptions/

In [10]:
!az aks show -n $AKS -g $RG 

{
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "availabilityZones": null,
      "count": 3,
      "enableAutoScaling": true,
      "enableNodePublicIp": null,
      "maxCount": 100,
      "maxPods": 110,
      "minCount": 3,
      "name": "nodepool1",
      "nodeLabels": null,
      "nodeTaints": null,
      "orchestratorVersion": "1.15.10",
      "osDiskSizeGb": 100,
      "osType": "Linux",
      "provisioningState": "Succeeded",
      "scaleSetEvictionPolicy": null,
      "scaleSetPriority": null,
      "tags": null,
      "type": "VirtualMachineScaleSets",
      "vmSize": "STANDARD_DS12_V2"
    }
  ],
  "apiServerAccessProfile": null,
  "dnsPrefix": "cody-image-cody-image-rg-656057",
  "enablePodSecurityPolicy": null,
  "enableRbac": true,
  "fqdn": "cody-image-cody-image-rg-656057-9c27a87c.hcp.westus2.azmk8s.io",
  "id": "/subscriptions/6560575d-fa06-4e7d-95fb-f962e74efd7a/resourcegroups/cody-image-rg/providers/Microsoft.ContainerService/mana

In [11]:
!az aks get-credentials -n $AKS -g $RG --overwrite-existing

Merged "cody-image" as current context in /home/azureuser/.kube/config


In [12]:
!kubectl --namespace kube-system create serviceaccount tiller

serviceaccount/tiller created


In [13]:
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller

clusterrolebinding.rbac.authorization.k8s.io/tiller created


In [14]:
!helm init --service-account tiller --history-max 100 --wait

$HELM_HOME has been configured at /home/azureuser/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation


In [15]:
!kubectl patch deployment tiller-deploy --namespace=kube-system --type=json --patch='[{"op": "add", "path": "/spec/template/spec/containers/0/command", "value": ["/tiller", "--listen=localhost:44134"]}]'

deployment.extensions/tiller-deploy patched


In [16]:
!helm version

Client: &version.Version{SemVer:"v2.16.3", GitCommit:"1ee0254c86d4ed6887327dabed7aa7da29d7eb0d", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.16.3", GitCommit:"1ee0254c86d4ed6887327dabed7aa7da29d7eb0d", GitTreeState:"clean"}


In [17]:
!helm repo add dask-gateway https://dask.org/dask-gateway-helm-repo

"dask-gateway" has been added to your repositories


In [18]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Skip local chart repository
...Successfully got an update from the "dask-gateway" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete.


In [19]:
import secrets

to_write = f"""
gateway:
  proxyToken: {secrets.token_hex(32)}
    
clusterManager:
  image:
    name: daskdev/dask
    pullPolicy: IfNotPresent

  scheduler:
    cores:
      request: 8
      limit: 8
    memory:
      request: 112G
      limit: 112G
            
  worker:
    cores:
      request: 8
      limit: 8  
    memory:
      request: 112G
      limit: 112G
""".lstrip()

with open('config.yaml', 'w') as f:
    f.write(to_write)

In [20]:
!helm upgrade --install --namespace $DG --version $DGV --values config.yaml $DG dask-gateway/dask-gateway

Release "dask-gateway" does not exist. Installing it now.
NAME:   dask-gateway
LAST DEPLOYED: Sun Mar 22 02:44:07 2020
NAMESPACE: dask-gateway
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME          AGE
dask-gateway  0s

==> v1/Deployment
NAME                          AGE
gateway-dask-gateway          0s
scheduler-proxy-dask-gateway  0s
web-proxy-dask-gateway        0s

==> v1/Pod(related)
NAME                                           AGE
gateway-dask-gateway-6f66d8d5f7-6nd8d          0s
scheduler-proxy-dask-gateway-5c88c8f676-xfzks  0s
web-proxy-dask-gateway-8588fdc49b-9prqh        0s

==> v1/Role
NAME          AGE
dask-gateway  0s

==> v1/RoleBinding
NAME          AGE
dask-gateway  0s

==> v1/Secret
NAME          AGE
dask-gateway  0s

==> v1/Service
NAME                           AGE
gateway-api-dask-gateway       0s
scheduler-api-dask-gateway     0s
scheduler-public-dask-gateway  0s
web-api-dask-gateway           0s
web-public-dask-gateway        0s

==> v1/ServiceAccount
NAME 

In [23]:
!kubectl get service --namespace dask-gateway

NAME                            TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)          AGE
gateway-api-dask-gateway        ClusterIP      10.0.15.25     <none>          8001/TCP         57s
scheduler-api-dask-gateway      ClusterIP      10.0.230.137   <none>          8001/TCP         57s
scheduler-public-dask-gateway   LoadBalancer   10.0.241.44    52.143.82.133   8786:31880/TCP   57s
web-api-dask-gateway            ClusterIP      10.0.212.209   <none>          8001/TCP         57s
web-public-dask-gateway         LoadBalancer   10.0.8.44      52.143.86.145   80:32597/TCP     57s


In [25]:
from dask_gateway import Gateway

In [28]:
gateway = Gateway('http://52.143.86.145', proxy_address='tls://52.143.82.133:8786')

In [29]:
gateway.list_clusters()

[]

In [30]:
cluster = gateway.new_cluster()

In [ ]:
help(gateway.new_cluster)

In [31]:
cluster

In [32]:
c = cluster.get_client()
c

Client Scheduler: gateway://52.143.82.133:8786/e2e5416d7cf14081bed1922e60060329 Dashboard: http://52.143.82.133:46481/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [33]:
cluster.scale(1)

In [34]:
cluster.scale(20)

In [35]:
options = gateway.cluster_options()

In [36]:
import dask

In [37]:
@dask.delayed
def testing():
    return 'hi'

In [38]:
testing().compute()

'hi'

In [39]:
cluster.shutdown()

In [ ]:
!az group delete -n $RG --no-wait -y

In [ ]:
from dask_kubernetes import KubeCluster, make_pod_spec

In [ ]:
pod_spec = make_pod_spec(image='daskdev/dask:latest',
                          memory_limit='1G', memory_request='1G',
                          cpu_limit=1, cpu_request=1,
                          env={'EXTRA_PIP_PACKAGES': 'dask-lightgbm'}
                        ) 

In [ ]:
cluster = KubeCluster(pod_spec)

In [ ]:
cluster

In [ ]:
cluster.scale(10)

In [ ]:
from dask.distributed import Client

In [ ]:
c = Client(cluster)

In [ ]:
c

In [ ]:
c.get_versions(check=True)

In [ ]:
import dask

In [ ]:
@dask.delayed
def testing():
    print('hi')

In [ ]:
testing().compute()

In [ ]:
cluster.close()